In [ ]:
# =============================================================================
# cell 1: environment setup (v11 - POCL + Channel-wise)
# =============================================================================
import os
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

import sys
import time
import math
import json
import base64
from pathlib import Path
from datetime import datetime
from dataclasses import dataclass, asdict
from typing import Dict, List, Optional, Tuple, Any
import warnings
warnings.filterwarnings('ignore')

# v11: torch.compile flag (disable if causing issues)
USE_TORCH_COMPILE = True
USE_GRADIENT_CHECKPOINTING = True

# generate timestamp for this run
RUN_TIMESTAMP = datetime.now().strftime('%Y-%m-%d_%H%M%S')
print(f"run timestamp: {RUN_TIMESTAMP}")

# detect platform
IS_KAGGLE = 'KAGGLE_KERNEL_RUN_TYPE' in os.environ
IS_COLAB = 'COLAB_GPU' in os.environ or 'google.colab' in sys.modules
PLATFORM = 'kaggle' if IS_KAGGLE else 'colab' if IS_COLAB else 'local'
OUTPUT_DIR = '/kaggle/working/outputs' if IS_KAGGLE else 'outputs'

for subdir in ['figures', 'checkpoints', 'logs', 'results']:
    os.makedirs(f'{OUTPUT_DIR}/{subdir}', exist_ok=True)

print(f"platform: {PLATFORM}")
print(f"output directory: {OUTPUT_DIR}")
print(f"torch.compile: {'enabled' if USE_TORCH_COMPILE else 'disabled'}")
print(f"gradient checkpointing: {'enabled' if USE_GRADIENT_CHECKPOINTING else 'disabled'}")

In [ ]:
# =============================================================================
# cell 2: pytorch and hardware setup (v11 - POCL)
# =============================================================================
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.checkpoint import checkpoint
import numpy as np

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

from tqdm.auto import tqdm

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
SEED = 42

torch.manual_seed(SEED)
np.random.seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True
    torch.backends.cudnn.benchmark = True

    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"gpu: {gpu_name}")
    print(f"memory: {gpu_memory:.1f} gb")

# v11: set float32 matmul precision for torch.compile
if USE_TORCH_COMPILE and hasattr(torch, 'set_float32_matmul_precision'):
    torch.set_float32_matmul_precision('high')
    print("float32 matmul precision: high (for torch.compile)")

print(f"device: {DEVICE}")
print(f"pytorch: {torch.__version__}")

# check torch.compile availability
TORCH_COMPILE_AVAILABLE = hasattr(torch, 'compile') and torch.__version__ >= '2.0'
print(f"torch.compile available: {TORCH_COMPILE_AVAILABLE}")

In [ ]:
# =============================================================================
# cell 4: configuration (v12.1 - PROPER CTKD with GRL)
# =============================================================================
@dataclass
class Config:
    # gpt-2 teacher (frozen, pre-trained)
    teacher_name: str = "gpt2"

    # student model architecture - v12.1: using v10 baseline (320d, 5L, ~22M)
    d_model: int = 320      # v10 value (DO NOT reduce)
    n_layers: int = 5       # v10 value (DO NOT reduce)
    vocab_size: int = 50257
    max_seq_len: int = 256

    # distillation training
    distill_steps: int = 3000
    distill_lr: float = 3e-4
    warmup_steps: int = 50        # minimal warmup for LR

    # v12.1: gradient accumulation (same as v10)
    accumulation_steps: int = 2   # effective batch = 8 * 2 = 16

    # v12.1: HIDDEN ALIGNMENT DISABLED (caused regression in v7)
    hidden_align_weight: float = 0.0   # DISABLED
    teacher_d_model: int = 768         # gpt-2 hidden dim
    teacher_n_layers: int = 12         # gpt-2 layers

    # ==========================================================================
    # v12.1: PROPER CTKD Configuration (with Gradient Reversal Layer)
    # ==========================================================================
    use_ctkd: bool = True                  # v12.1: ENABLED (proper CTKD with GRL)
    
    # Temperature bounds (conservative for LLMs - paper uses [1,21] for images)
    tau_min: float = 1.0                   # Minimum temperature
    tau_max: float = 5.0                   # Maximum temperature (5.0 is conservative)
    tau_init: float = 2.0                  # Initial temperature
    
    # Lambda (adversarial strength) scheduling
    lambda_max: float = 1.0                # Maximum adversarial strength
    lambda_warmup_ratio: float = 0.2       # 20% warmup (λ=0 for first 600 steps)
    
    # Legacy flags (disabled)
    use_learnable_temperature: bool = False  # DEPRECATED: Use use_ctkd instead
    use_channel_wise_spikes: bool = False    # DISABLED (structural symmetry issue)
    use_progressive_stages: bool = False     # DISABLED
    temperature: float = 2.0                 # Legacy, used if CTKD disabled
    temperature_lr: float = 0.001            # Legacy, not used with CTKD

    # lora for ttt
    lora_rank: int = 8
    lora_alpha: float = 16.0
    ttt_lr: float = 1e-4
    ttt_steps: int = 100

    # spiking parameters
    spike_alpha: float = 1.0

    # general training
    batch_size: int = 8
    max_grad_norm: float = 1.0
    eval_interval: int = 300  # reduced from 100 for speed

config = Config()

print(f"configuration (v12.1 - PROPER CTKD with Gradient Reversal Layer):")
print(f"  teacher: {config.teacher_name} (124m params)")
print(f"  student: d={config.d_model}, layers={config.n_layers}")
print(f"")
print(f"v12.1 INNOVATION - Proper CTKD (ArXiv 2211.16231):")
print(f"  use_ctkd: {config.use_ctkd} <-- v12.1 ENABLED")
print(f"")
print(f"  CTKD uses Gradient Reversal Layer (GRL) for ADVERSARIAL temperature:")
print(f"    - Student MINIMIZES KL loss (normal gradients)")
print(f"    - Temperature MAXIMIZES KL loss (reversed gradients)")
print(f"    - Result: Temperature finds OPTIMAL difficulty, not 'easy' solution")
print(f"")
print(f"  Temperature bounds (sigmoid, smooth gradients):")
print(f"    - τ_min: {config.tau_min}")
print(f"    - τ_max: {config.tau_max} (conservative for LLMs)")
print(f"    - τ_init: {config.tau_init}")
print(f"")
print(f"  Lambda schedule (adversarial strength):")
print(f"    - λ_max: {config.lambda_max}")
print(f"    - warmup: {config.lambda_warmup_ratio*100:.0f}% ({int(config.distill_steps * config.lambda_warmup_ratio)} steps)")
print(f"    - Schedule: cosine 0 → {config.lambda_max}")
print(f"")
print(f"v12.1 vs v12 (broken):")
print(f"  v12:   Both minimize → T finds 'easy' solution → runaway")
print(f"  v12.1: Min-max (GRL) → T finds 'hardest' difficulty → proper learning")
print(f"")
print(f"disabled features:")
print(f"  channel-wise spikes: {config.use_channel_wise_spikes} (structural symmetry issue)")
print(f"  progressive stages: {config.use_progressive_stages}")
print(f"  hidden alignment: {config.hidden_align_weight}")
print(f"")
print(f"training:")
print(f"  distillation: {config.distill_steps} steps")
print(f"  warmup (LR): {config.warmup_steps} steps")
print(f"  accumulation: {config.accumulation_steps} (effective batch = {config.batch_size * config.accumulation_steps})")
print(f"")
print(f"targets:")
print(f"  PPL: <500 (improve on v10's 514.5)")
print(f"  Temperature: should evolve meaningfully (not stuck at init or bounds)")

In [ ]:
# =============================================================================
# cell 6: v12.1 PROPER CTKD Implementation
# =============================================================================
# References:
# - CTKD Paper: https://arxiv.org/abs/2211.16231
# - GRL Origin: Ganin & Lempitsky (2015) https://arxiv.org/abs/1409.7495
# - torch-gradient-reversal: https://pypi.org/project/torch-gradient-reversal/

# -----------------------------------------------------------------------------
# GradientReversalFunction (Custom Autograd)
# -----------------------------------------------------------------------------
class GradientReversalFunction(torch.autograd.Function):
    """
    Gradient Reversal Layer for adversarial training.
    
    Forward: Identity mapping f(x) = x
    Backward: Negates gradient ∂f/∂x = -λ * grad
    
    This enables min-max optimization in a single backward pass:
    - Student minimizes loss (normal gradients)
    - Temperature maximizes loss (reversed gradients via GRL)
    
    Reference: Ganin & Lempitsky, "Unsupervised Domain Adaptation by Backpropagation"
    """
    
    @staticmethod
    def forward(ctx, x, lambda_):
        # Save lambda for backward pass
        ctx.lambda_ = lambda_
        # Forward is identity (must clone to avoid in-place issues)
        return x.clone()
    
    @staticmethod
    def backward(ctx, grad_output):
        # Backward negates and scales gradient
        # Returns: (grad for x, grad for lambda_)
        # lambda_ is a hyperparameter, doesn't need gradient
        return -ctx.lambda_ * grad_output, None


class GradientReversalLayer(nn.Module):
    """
    Module wrapper for GradientReversalFunction.
    
    Usage:
        grl = GradientReversalLayer()
        grl.set_lambda(0.5)  # Set adversarial strength
        y = grl(x)  # Forward: y = x, Backward: grad_x = -0.5 * grad_y
    """
    
    def __init__(self):
        super().__init__()
        self.lambda_ = 1.0
    
    def set_lambda(self, lambda_: float):
        """Set the adversarial strength (0 = no reversal, 1 = full reversal)."""
        self.lambda_ = lambda_
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return GradientReversalFunction.apply(x, self.lambda_)


# -----------------------------------------------------------------------------
# Lambda Scheduler (Cosine with Warmup)
# -----------------------------------------------------------------------------
def get_lambda(step: int, total_steps: int, lambda_max: float = 1.0, 
               warmup_ratio: float = 0.2) -> float:
    """
    Cosine schedule for adversarial strength λ.
    
    - During warmup (first warmup_ratio of training): λ = 0
      Temperature learns freely to find reasonable range
    - After warmup: λ increases from 0 to lambda_max via cosine
      Gradually increases adversarial pressure
    
    Args:
        step: Current training step
        total_steps: Total number of training steps
        lambda_max: Maximum λ value (default 1.0 = full reversal)
        warmup_ratio: Fraction of training for warmup (default 0.2 = 20%)
    
    Returns:
        Current λ value in [0, lambda_max]
    """
    warmup_steps = int(total_steps * warmup_ratio)
    
    if step < warmup_steps:
        return 0.0
    
    # Progress after warmup [0, 1]
    progress = (step - warmup_steps) / max(total_steps - warmup_steps, 1)
    # Cosine increase from 0 to lambda_max
    lambda_ = lambda_max * (1 - math.cos(math.pi * progress)) / 2
    return lambda_


# -----------------------------------------------------------------------------
# CTKDTemperature (Proper Implementation with GRL)
# -----------------------------------------------------------------------------
class CTKDTemperature(nn.Module):
    """
    Curriculum Temperature for Knowledge Distillation (CTKD).
    
    Key features:
    1. Adversarial learning via Gradient Reversal Layer
    2. Sigmoid bounding for smooth gradients at boundaries
    3. Proper initialization via logit transform
    
    The temperature module tries to MAXIMIZE the KL loss (via GRL),
    finding the "hardest" temperature for the student.
    The student tries to MINIMIZE the KL loss.
    This adversarial game leads to optimal curriculum difficulty.
    
    Reference: Li et al., "Curriculum Temperature for Knowledge Distillation", AAAI 2023
    """
    
    def __init__(self, tau_min: float = 1.0, tau_max: float = 5.0, init: float = 2.0):
        """
        Args:
            tau_min: Minimum temperature (default 1.0)
            tau_max: Maximum temperature (default 5.0, conservative for LLMs)
            init: Initial temperature (default 2.0)
        """
        super().__init__()
        self.tau_min = tau_min
        self.tau_range = tau_max - tau_min
        
        # Initialize raw parameter so sigmoid outputs init value
        # sigmoid(raw) = (init - tau_min) / tau_range
        # raw = logit((init - tau_min) / tau_range)
        init_normalized = (init - tau_min) / self.tau_range
        init_normalized = max(0.01, min(0.99, init_normalized))  # Clamp for numerical stability
        init_raw = math.log(init_normalized / (1 - init_normalized))  # logit function
        
        self.raw_temp = nn.Parameter(torch.tensor(init_raw, dtype=torch.float32))
        self.grl = GradientReversalLayer()
        
        # Store config for logging
        self.tau_min_val = tau_min
        self.tau_max_val = tau_max
        self.init_val = init
    
    def forward(self, lambda_: float) -> torch.Tensor:
        """
        Compute temperature with GRL applied.
        
        Args:
            lambda_: Current adversarial strength from scheduler
        
        Returns:
            Temperature τ ∈ [tau_min, tau_max]
        """
        # Set GRL strength
        self.grl.set_lambda(lambda_)
        
        # Apply GRL to raw parameter (this is where gradient reversal happens!)
        raw_reversed = self.grl(self.raw_temp)
        
        # Sigmoid bounding (smooth, differentiable at boundaries)
        tau = self.tau_min + self.tau_range * torch.sigmoid(raw_reversed)
        
        return tau
    
    def get_temperature(self) -> float:
        """Get current temperature without GRL (for logging/display)."""
        with torch.no_grad():
            tau = self.tau_min + self.tau_range * torch.sigmoid(self.raw_temp)
            return tau.item()
    
    def get_raw_value(self) -> float:
        """Get raw (unbounded) parameter value (for debugging)."""
        return self.raw_temp.item()


# -----------------------------------------------------------------------------
# Legacy Classes (kept for backward compatibility)
# -----------------------------------------------------------------------------
class LearnableTemperature(nn.Module):
    """
    DEPRECATED: Simple learnable temperature WITHOUT GRL.
    Kept for backward compatibility. Use CTKDTemperature instead.
    
    WARNING: This class caused temperature runaway in v12!
    """
    
    def __init__(self, init: float = 2.0):
        super().__init__()
        self.log_temp = nn.Parameter(torch.log(torch.tensor(init)))
    
    def forward(self) -> torch.Tensor:
        return torch.exp(self.log_temp).clamp(1.0, 10.0)
    
    def get_temperature(self) -> float:
        return self.forward().item()


class ChannelWiseTernarySpike(nn.Module):
    """
    Per-channel learnable alpha and amplitude for ternary spikes.
    DISABLED in v12.1 due to structural symmetry issue with RWKV.
    """
    
    def __init__(self, d_model: int, alpha_init: float = 1.0):
        super().__init__()
        self.d_model = d_model
        self.alpha = nn.Parameter(torch.ones(d_model) * alpha_init)
        self.amplitude = nn.Parameter(torch.ones(d_model))
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x_abs_mean = x.abs().mean(dim=(0, 1), keepdim=True)
        threshold = self.alpha * x_abs_mean
        threshold = threshold.clamp(min=0.01, max=10.0)
        
        with torch.no_grad():
            pos_mask = (x > threshold).float()
            neg_mask = (x < -threshold).float()
            spike_signs = pos_mask - neg_mask
        
        spikes = self.amplitude * spike_signs
        return spikes + (x - x.detach())
    
    def get_amplitude(self) -> float:
        return self.amplitude.mean().item()
    
    def get_stats(self) -> dict:
        return {
            'alpha_mean': self.alpha.mean().item(),
            'alpha_std': self.alpha.std().item(),
            'amplitude_mean': self.amplitude.mean().item(),
            'amplitude_std': self.amplitude.std().item(),
        }

    def get_amplitude_stats(self) -> dict:
        return {
            'mean': self.amplitude.mean().item(),
            'std': self.amplitude.std().item(),
            'min': self.amplitude.min().item(),
            'max': self.amplitude.max().item(),
        }


class TrainableTernarySpike(nn.Module):
    """Original trainable ternary spike with scalar amplitude (from v8)."""

    def __init__(self, alpha: float = 1.0):
        super().__init__()
        self.alpha = alpha
        self.amplitude = nn.Parameter(torch.ones(1))

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        threshold = self.alpha * x.abs().mean(dim=-1, keepdim=True)
        threshold = threshold.clamp(min=0.01, max=10.0)

        with torch.no_grad():
            pos_mask = (x > threshold).float()
            neg_mask = (x < -threshold).float()
            spike_signs = pos_mask - neg_mask

        spikes = self.amplitude * spike_signs
        return spikes + (x - x.detach())

    def get_amplitude(self) -> float:
        return self.amplitude.item()


def get_stage_params(step: int, total_steps: int = 3000) -> dict:
    """Progressive training stages (POCL) - kept for infrastructure."""
    if step < total_steps * 0.4:
        return {'stage': 1, 'temp_target': 1.0, 'align_mult': 0.0, 'alpha': 0.9}
    elif step < total_steps * 0.7:
        return {'stage': 2, 'temp_target': 1.5, 'align_mult': 0.5, 'alpha': 0.7}
    else:
        return {'stage': 3, 'temp_target': 2.0, 'align_mult': 1.0, 'alpha': 0.5}


# -----------------------------------------------------------------------------
# Unit Tests for CTKD Components
# -----------------------------------------------------------------------------
print("="*60)
print("v12.1 CTKD Component Tests")
print("="*60)

# Test 1: GRL Gradient Reversal
print("\n[1] GRL Gradient Reversal Test")
grl = GradientReversalLayer()
grl.set_lambda(1.0)
x_test = torch.tensor([2.0], requires_grad=True)
y_test = grl(x_test)
loss_test = y_test.sum()
loss_test.backward()
expected_grad = -1.0  # GRL should negate: 1 * -1.0 = -1.0
actual_grad = x_test.grad.item()
grl_pass = abs(actual_grad - expected_grad) < 1e-6
print(f"  Input grad without GRL would be: +1.0")
print(f"  Input grad with GRL (λ=1.0): {actual_grad:.4f}")
print(f"  Expected: {expected_grad:.4f}")
print(f"  {'PASS' if grl_pass else 'FAIL'}")
del x_test, y_test, loss_test

# Test 2: Lambda Schedule
print("\n[2] Lambda Schedule Test")
total = 3000
warmup = 0.2
# During warmup
lambda_0 = get_lambda(0, total, warmup_ratio=warmup)
lambda_500 = get_lambda(500, total, warmup_ratio=warmup)
# After warmup
lambda_1500 = get_lambda(1500, total, warmup_ratio=warmup)
lambda_2999 = get_lambda(2999, total, warmup_ratio=warmup)

warmup_pass = lambda_0 == 0.0 and lambda_500 == 0.0
increase_pass = 0 < lambda_1500 < lambda_2999 <= 1.0
lambda_pass = warmup_pass and increase_pass
print(f"  λ(0) = {lambda_0:.4f} (should be 0.0)")
print(f"  λ(500) = {lambda_500:.4f} (should be 0.0, still in warmup)")
print(f"  λ(1500) = {lambda_1500:.4f} (should be > 0)")
print(f"  λ(2999) = {lambda_2999:.4f} (should be ≈ 1.0)")
print(f"  {'PASS' if lambda_pass else 'FAIL'}")

# Test 3: Temperature Bounds
print("\n[3] Temperature Bounds Test")
temp_module = CTKDTemperature(tau_min=1.0, tau_max=5.0, init=2.0).to(DEVICE)
init_temp = temp_module.get_temperature()

# Force extreme raw values
with torch.no_grad():
    temp_module.raw_temp.fill_(-100)
    tau_low = temp_module.get_temperature()
    
    temp_module.raw_temp.fill_(100)
    tau_high = temp_module.get_temperature()
    
    # Reset to init
    init_normalized = (2.0 - 1.0) / 4.0
    init_raw = math.log(init_normalized / (1 - init_normalized))
    temp_module.raw_temp.fill_(init_raw)

bounds_pass = (1.0 <= tau_low <= 1.01) and (4.99 <= tau_high <= 5.0) and (1.9 <= init_temp <= 2.1)
print(f"  Initial temp: {init_temp:.4f} (should be ≈ 2.0)")
print(f"  Min bound test: {tau_low:.4f} (should be ≈ 1.0)")
print(f"  Max bound test: {tau_high:.4f} (should be ≈ 5.0)")
print(f"  {'PASS' if bounds_pass else 'FAIL'}")

# Test 4: End-to-End Gradient Flow
print("\n[4] End-to-End Gradient Flow Test")
temp_module_test = CTKDTemperature(tau_min=1.0, tau_max=5.0, init=2.0).to(DEVICE)
lambda_test = 0.5

# Simulate forward pass
T = temp_module_test(lambda_test)
fake_kl_loss = T * 2.0  # Gradient ∂L/∂T = 2.0

# Without GRL: optimizer would DECREASE T to minimize loss
# With GRL: optimizer should INCREASE T (because grad is reversed)
fake_kl_loss.backward()

raw_grad = temp_module_test.raw_temp.grad.item()
# The gradient through sigmoid and GRL should be negative (reversed)
# Original: ∂L/∂raw > 0 would decrease raw
# With GRL: ∂L/∂raw < 0 (negated), so optimizer increases raw
grad_flow_pass = raw_grad < 0  # Should be negative due to GRL
print(f"  Loss = T * 2.0, so ∂L/∂T = 2.0 (positive)")
print(f"  Without GRL: raw_grad would be positive (decrease T)")
print(f"  With GRL (λ=0.5): raw_grad = {raw_grad:.4f} (should be negative)")
print(f"  {'PASS' if grad_flow_pass else 'FAIL'}")
del temp_module_test

# Summary
print("\n" + "="*60)
all_pass = grl_pass and lambda_pass and bounds_pass and grad_flow_pass
print(f"CTKD Component Tests: {'ALL PASS' if all_pass else 'SOME FAILED'}")
if not all_pass:
    print("WARNING: Fix failing tests before running training!")
print("="*60)

In [ ]:
# =============================================================================
# cell 7: hardware and spike stats collectors (same as v9)
# =============================================================================
class HardwareStatsCollector:
    """collect gpu memory, timing, and throughput metrics."""

    def __init__(self):
        self.gpu_memory_history = []
        self.step_times = []
        self.tokens_processed = 0
        self.start_time = None

    def start(self):
        self.start_time = time.time()
        if torch.cuda.is_available():
            torch.cuda.reset_peak_memory_stats()

    def record_step(self, batch_size: int, seq_len: int):
        if torch.cuda.is_available():
            self.gpu_memory_history.append(torch.cuda.memory_allocated() / 1e9)
        self.tokens_processed += batch_size * seq_len
        self.step_times.append(time.time())

    def get_throughput(self) -> float:
        if len(self.step_times) < 2:
            return 0.0
        elapsed = self.step_times[-1] - self.step_times[0]
        return self.tokens_processed / elapsed if elapsed > 0 else 0.0

    def get_summary(self) -> Dict[str, Any]:
        elapsed = time.time() - self.start_time if self.start_time else 0
        return {
            'peak_gpu_memory_gb': max(self.gpu_memory_history) if self.gpu_memory_history else 0,
            'avg_gpu_memory_gb': float(np.mean(self.gpu_memory_history)) if self.gpu_memory_history else 0,
            'total_training_time_s': elapsed,
            'total_training_time_min': elapsed / 60,
            'tokens_processed': self.tokens_processed,
            'throughput_tokens_per_sec': self.get_throughput(),
        }


class SpikeStatsCollector:
    """collect per-layer spike density and amplitude evolution."""

    def __init__(self, n_layers: int):
        self.n_layers = n_layers
        self.density_history = {i: {'k': [], 'v': []} for i in range(n_layers)}
        self.amplitude_history = {i: {'k': [], 'v': []} for i in range(n_layers)}
        self.step_densities = []

    def record(self, student, step: int):
        stats = student.get_spike_stats()
        all_densities = []
        for i in range(self.n_layers):
            layer_key = f'layer_{i}'
            if layer_key in stats:
                k_density = stats[layer_key].get('k', 0)
                v_density = stats[layer_key].get('v', 0)
                k_amp = stats[layer_key].get('k_amp', 1.0)
                v_amp = stats[layer_key].get('v_amp', 1.0)

                self.density_history[i]['k'].append(k_density)
                self.density_history[i]['v'].append(v_density)
                self.amplitude_history[i]['k'].append(k_amp)
                self.amplitude_history[i]['v'].append(v_amp)
                all_densities.extend([k_density, v_density])

        if all_densities:
            self.step_densities.append({'step': step, 'density': float(np.mean(all_densities))})

    def get_summary(self) -> Dict[str, Any]:
        per_layer = {}
        all_k, all_v = [], []
        all_k_amp, all_v_amp = [], []

        for i in range(self.n_layers):
            k_vals = self.density_history[i]['k']
            v_vals = self.density_history[i]['v']
            k_amps = self.amplitude_history[i]['k']
            v_amps = self.amplitude_history[i]['v']

            per_layer[f'layer_{i}'] = {
                'k_mean': float(np.mean(k_vals)) if k_vals else 0,
                'k_std': float(np.std(k_vals)) if k_vals else 0,
                'k_final': float(k_vals[-1]) if k_vals else 0,
                'v_mean': float(np.mean(v_vals)) if v_vals else 0,
                'v_std': float(np.std(v_vals)) if v_vals else 0,
                'v_final': float(v_vals[-1]) if v_vals else 0,
                'k_amp_final': float(k_amps[-1]) if k_amps else 1.0,
                'v_amp_final': float(v_amps[-1]) if v_amps else 1.0,
            }
            all_k.extend(k_vals)
            all_v.extend(v_vals)
            if k_amps: all_k_amp.append(k_amps[-1])
            if v_amps: all_v_amp.append(v_amps[-1])

        return {
            'per_layer': per_layer,
            'overall_k_density': float(np.mean(all_k)) if all_k else 0,
            'overall_v_density': float(np.mean(all_v)) if all_v else 0,
            'overall_density': float(np.mean(all_k + all_v)) if (all_k or all_v) else 0,
            'amplitudes': {'k': all_k_amp, 'v': all_v_amp},
            'density_history': self.step_densities,
        }

print("collectors defined")

In [ ]:
# =============================================================================
# cell 8: spiking goose model (v11 - channel-wise spikes + gradient checkpointing)
# =============================================================================
class SpikingGooseRecurrentLayer(nn.Module):
    """
    RWKV-style recurrence with trainable ternary spiking.
    
    v11: Supports channel-wise ternary spikes (when use_channel_wise=True)
    """

    def __init__(self, d_model, layer_idx=0, n_layers=4, spike_alpha=1.0, 
                 use_channel_wise: bool = False):
        super().__init__()
        self.d_model = d_model
        self.layer_idx = layer_idx
        self.use_channel_wise = use_channel_wise
        self.ln = nn.LayerNorm(d_model)

        ratio = layer_idx / max(n_layers - 1, 1)
        self.time_mix_k = nn.Parameter(torch.ones(d_model) * (1 - ratio))
        self.time_mix_v = nn.Parameter(torch.ones(d_model) * (1 - ratio))
        self.time_mix_r = nn.Parameter(torch.ones(d_model) * (1 - ratio))
        self.decay_weight = nn.Parameter(torch.zeros(d_model) - 0.5)

        self.key_proj = nn.Linear(d_model, d_model, bias=False)
        self.value_proj = nn.Linear(d_model, d_model, bias=False)
        self.receptance_proj = nn.Linear(d_model, d_model, bias=False)
        self.output_proj = nn.Linear(d_model, d_model, bias=False)

        # v11: Use channel-wise spikes if enabled
        if use_channel_wise:
            self.k_spike = ChannelWiseTernarySpike(d_model, alpha_init=spike_alpha)
            self.v_spike = ChannelWiseTernarySpike(d_model, alpha_init=spike_alpha)
        else:
            self.k_spike = TrainableTernarySpike(alpha=spike_alpha)
            self.v_spike = TrainableTernarySpike(alpha=spike_alpha)

        self.register_buffer('running_k_density', torch.tensor(0.0))
        self.register_buffer('running_v_density', torch.tensor(0.0))
        self._init_weights()

    def _init_weights(self):
        std = 0.1 / math.sqrt(self.d_model)
        for m in [self.key_proj, self.value_proj, self.receptance_proj, self.output_proj]:
            nn.init.normal_(m.weight, std=std)

    def forward(self, x):
        B, T, D = x.shape
        x_norm = self.ln(x)
        prev_x = F.pad(x_norm[:, :-1, :], (0, 0, 1, 0))

        xk = x_norm * self.time_mix_k + prev_x * (1 - self.time_mix_k)
        xv = x_norm * self.time_mix_v + prev_x * (1 - self.time_mix_v)
        xr = x_norm * self.time_mix_r + prev_x * (1 - self.time_mix_r)

        k_pre = self.key_proj(xk)
        v_pre = self.value_proj(xv)

        k = self.k_spike(k_pre)
        v = self.v_spike(v_pre)
        r = torch.sigmoid(self.receptance_proj(xr))

        kv = k * v
        decay = torch.sigmoid(self.decay_weight)
        t_idx = torch.arange(T, device=x.device, dtype=x.dtype)
        decay_powers = decay.unsqueeze(0) ** t_idx.unsqueeze(1)

        kv_weighted = kv / (decay_powers.unsqueeze(0) + 1e-8)
        S = torch.cumsum(kv_weighted, dim=1) * decay_powers.unsqueeze(0)

        if self.training:
            with torch.no_grad():
                self.running_k_density = 0.99 * self.running_k_density + 0.01 * (k != 0).float().mean()
                self.running_v_density = 0.99 * self.running_v_density + 0.01 * (v != 0).float().mean()

        return x + r * self.output_proj(S)

    def get_spike_density(self):
        return {
            'k': self.running_k_density.item(),
            'v': self.running_v_density.item(),
            'k_amp': self.k_spike.get_amplitude(),
            'v_amp': self.v_spike.get_amplitude(),
        }
    
    def get_channel_wise_stats(self) -> dict:
        """Get channel-wise spike statistics (only available if use_channel_wise=True)."""
        if self.use_channel_wise:
            return {
                'k': self.k_spike.get_stats(),
                'v': self.v_spike.get_stats(),
            }
        return None


class GooseFFN(nn.Module):
    def __init__(self, d_model, expand=4):
        super().__init__()
        self.ln = nn.LayerNorm(d_model)
        self.w1 = nn.Linear(d_model, d_model * expand, bias=False)
        self.w2 = nn.Linear(d_model * expand, d_model, bias=False)

    def forward(self, x):
        return x + self.w2(F.silu(self.w1(self.ln(x))))


class StudentSpikingGoose(nn.Module):
    """
    Spiking student model with trainable ternary activations.
    
    v11: Supports channel-wise ternary spikes + gradient checkpointing.
    """

    def __init__(self, cfg, use_checkpointing=True):
        super().__init__()
        self.cfg = cfg
        self.use_checkpointing = use_checkpointing and USE_GRADIENT_CHECKPOINTING
        
        # v11: Check for channel-wise spikes flag
        use_channel_wise = getattr(cfg, 'use_channel_wise_spikes', False)
        
        self.embed = nn.Embedding(cfg.vocab_size, cfg.d_model)
        self.pos_embed = nn.Embedding(cfg.max_seq_len, cfg.d_model)

        self.layers = nn.ModuleList([
            nn.ModuleDict({
                'rec': SpikingGooseRecurrentLayer(
                    cfg.d_model, i, cfg.n_layers, cfg.spike_alpha,
                    use_channel_wise=use_channel_wise
                ),
                'ffn': GooseFFN(cfg.d_model),
            })
            for i in range(cfg.n_layers)
        ])

        self.ln_out = nn.LayerNorm(cfg.d_model)
        self.head = nn.Linear(cfg.d_model, cfg.vocab_size, bias=False)
        self.head.weight = self.embed.weight

        nn.init.normal_(self.embed.weight, std=0.02)
        nn.init.normal_(self.pos_embed.weight, std=0.02)

    def _layer_forward(self, layer, x):
        """helper for gradient checkpointing - processes one layer."""
        x = layer['rec'](x)
        x = layer['ffn'](x)
        return x

    def forward(self, input_ids, return_hiddens=False):
        """forward pass with optional hidden state return for alignment."""
        B, T = input_ids.shape
        pos = torch.arange(T, device=input_ids.device).unsqueeze(0)
        x = self.embed(input_ids) + self.pos_embed(pos)

        hiddens = [x] if return_hiddens else None

        for layer in self.layers:
            if self.use_checkpointing and self.training:
                # v10: gradient checkpointing with use_reentrant=False (recommended)
                x = checkpoint(self._layer_forward, layer, x, use_reentrant=False)
            else:
                x = self._layer_forward(layer, x)
            
            if return_hiddens:
                hiddens.append(x)

        logits = self.head(self.ln_out(x))

        if return_hiddens:
            return logits, hiddens
        return logits

    def get_spike_stats(self):
        return {f'layer_{i}': layer['rec'].get_spike_density() for i, layer in enumerate(self.layers)}

    def get_avg_spike_density(self):
        densities = []
        for layer in self.layers:
            d = layer['rec'].get_spike_density()
            densities.extend([d['k'], d['v']])
        return float(np.mean(densities)) if densities else 0.0

    def get_amplitudes(self):
        return {f'layer_{i}': {'k': layer['rec'].k_spike.get_amplitude(), 'v': layer['rec'].v_spike.get_amplitude()}
                for i, layer in enumerate(self.layers)}
    
    def get_channel_amplitude_variance(self) -> float:
        """Get total variance of channel-wise amplitudes (for regularization)."""
        total_var = 0.0
        for layer in self.layers:
            rec = layer['rec']
            if hasattr(rec.k_spike, 'amplitude') and rec.k_spike.amplitude.numel() > 1:
                total_var += rec.k_spike.amplitude.var().item()
                total_var += rec.v_spike.amplitude.var().item()
        return total_var

print("student model defined (v11: channel-wise spikes + gradient checkpointing)")
print(f"  gradient checkpointing: {USE_GRADIENT_CHECKPOINTING}")
print(f"  channel-wise spikes: {config.use_channel_wise_spikes}")


In [ ]:
# =============================================================================
# cell 9: hidden-state alignment (v11 - 320d × 5L, but DISABLED)
# =============================================================================
class HiddenStateProjector(nn.Module):
    """
    Project student hidden states to teacher dimension for alignment.

    student: (B, T, 512) -> (B, T, 768)  # v11: 512 dim (same as v11)

    Maps 8 student layers to selected teacher layers.

    NOTE: This is kept for infrastructure but DISABLED in v11 (weight=0.0).
    """

    def __init__(self, student_dim: int, teacher_dim: int, n_student_layers: int):
        super().__init__()
        self.projectors = nn.ModuleList([
            nn.Linear(student_dim, teacher_dim, bias=False)
            for _ in range(n_student_layers)
        ])
        for proj in self.projectors:
            nn.init.normal_(proj.weight, std=0.02)

    def forward(self, student_hidden: torch.Tensor, layer_idx: int) -> torch.Tensor:
        return self.projectors[layer_idx](student_hidden)


def compute_hidden_alignment_loss(
    teacher_hiddens: List[torch.Tensor],
    student_hiddens: List[torch.Tensor],
    projector: HiddenStateProjector,
    teacher_layers: int = 12,
    student_layers: int = 8
) -> torch.Tensor:
    """
    Compute MSE loss between projected student and teacher hidden states.

    v11: Uses 8 student layers, maps to every 1-2 teacher layers.
    NOTE: Disabled in v11 (hidden_align_weight=0.0), kept for future experiments.
    """
    # v11: Map 8 student layers to 8 teacher layers
    # teacher_indices[i] = which teacher layer student layer i maps to
    teacher_indices = [1, 2, 4, 5, 7, 8, 10, 11]  # v11 mapping (8 student layers)

    total_loss = 0.0
    for s_idx, t_idx in enumerate(teacher_indices):
        if s_idx >= len(student_hiddens) - 1:  # student_hiddens includes embedding
            break
        if t_idx >= len(teacher_hiddens):
            break

        s_hidden = student_hiddens[s_idx + 1]  # +1 because [0] is embedding
        t_hidden = teacher_hiddens[t_idx]

        s_proj = projector(s_hidden, s_idx)
        total_loss = total_loss + F.mse_loss(s_proj, t_hidden)

    return total_loss / len(teacher_indices)


# Create projector (even if disabled, keeps infrastructure)
projector = HiddenStateProjector(
    student_dim=config.d_model,
    teacher_dim=config.teacher_d_model,
    n_student_layers=config.n_layers
).to(DEVICE)

projector_params = sum(p.numel() for p in projector.parameters())
print(f"hidden-state projector: {projector_params:,} params")
print(f"  student dim: {config.d_model} (v11: same as v11)")
print(f"  teacher dim: {config.teacher_d_model}")
print(f"  student layers: {config.n_layers}")
print(f"  hidden_align_weight: {config.hidden_align_weight}")
if config.hidden_align_weight == 0.0:
    print(f"  STATUS: DISABLED (keeping infrastructure for future experiments)")

In [ ]:
# =============================================================================
# cell 10: cosine lr with warmup (same as v9)
# =============================================================================
def get_cosine_schedule_with_warmup(
    optimizer: torch.optim.Optimizer,
    warmup_steps: int,
    total_steps: int,
) -> torch.optim.lr_scheduler.LambdaLR:
    """
    linear warmup then cosine decay to 0.
    """
    def lr_lambda(step: int) -> float:
        if step < warmup_steps:
            return step / max(warmup_steps, 1)
        else:
            progress = (step - warmup_steps) / max(total_steps - warmup_steps, 1)
            return 0.5 * (1.0 + math.cos(math.pi * progress))

    return torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)


print(f"cosine lr: {config.warmup_steps} warmup, {config.distill_steps} total")

In [ ]:
# =============================================================================
# cell 11: load gpt-2 teacher (same as v9)
# =============================================================================
from transformers import GPT2LMHeadModel, GPT2Tokenizer

print("loading gpt-2 teacher...")
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

teacher = GPT2LMHeadModel.from_pretrained('gpt2').to(DEVICE)
teacher.config.use_cache = False  # Disable KV caching (not needed for distillation)

# Compile teacher for faster inference (PyTorch 2.0+)
try:
    teacher = torch.compile(teacher, mode='reduce-overhead')
    print('teacher compiled with torch.compile')
except Exception as e:
    print(f'torch.compile not available: {e}')
teacher.eval()
for p in teacher.parameters():
    p.requires_grad = False

teacher_params = sum(p.numel() for p in teacher.parameters())
print(f"teacher: gpt-2 ({teacher_params:,} params)")

In [ ]:
# =============================================================================
# cell 13: data loading (v11 - efficient DataLoader)
# =============================================================================
from datasets import load_dataset

print("loading wikitext-2...")
dataset = load_dataset('wikitext', 'wikitext-2-raw-v1')

def pre_tokenize(texts, max_len):
    all_tokens = []
    for text in tqdm(texts, desc="tokenizing", leave=False):
        if text.strip():
            all_tokens.extend(tokenizer.encode(text, max_length=max_len*2, truncation=True))
    chunks = [all_tokens[i:i+max_len] for i in range(0, len(all_tokens)-max_len+1, max_len//2) if len(all_tokens[i:i+max_len]) == max_len]
    print(f"created {len(chunks)} sequences")
    return torch.tensor(chunks, dtype=torch.long)

train_tokens = pre_tokenize(dataset['train']['text'], config.max_seq_len)
val_tokens = pre_tokenize(dataset['validation']['text'], config.max_seq_len)

# v11: efficient DataLoader with workers and prefetch
# Note: num_workers=0 for Kaggle/Colab compatibility, but prefetch still helps
dataloader_kwargs = {
    'batch_size': config.batch_size,
    'pin_memory': True,
    'num_workers': 0 if IS_KAGGLE or IS_COLAB else 2,  # workers disabled on cloud platforms
    'prefetch_factor': None if IS_KAGGLE or IS_COLAB else 2,
    'persistent_workers': False if IS_KAGGLE or IS_COLAB else True,
}

train_loader = DataLoader(TensorDataset(train_tokens), shuffle=True, **dataloader_kwargs)
val_loader = DataLoader(TensorDataset(val_tokens), shuffle=False, **dataloader_kwargs)

print(f"train: {len(train_loader)} batches, val: {len(val_loader)} batches")
print(f"DataLoader: num_workers={dataloader_kwargs['num_workers']}, pin_memory={dataloader_kwargs['pin_memory']}")

In [ ]:
# =============================================================================
# cell 14: create student model and projector (v11 - with compile)
# =============================================================================
print("creating student model (v11 - v11 baseline + POCL)...")

student = StudentSpikingGoose(config, use_checkpointing=USE_GRADIENT_CHECKPOINTING).to(DEVICE)
student_params = sum(p.numel() for p in student.parameters())

# v11: create projector (even if not used, for infrastructure preservation)
projector = HiddenStateProjector(
    student_dim=config.d_model,
    teacher_dim=config.teacher_d_model,
    n_student_layers=config.n_layers
).to(DEVICE)
projector_params = sum(p.numel() for p in projector.parameters())

compression_ratio = teacher_params / student_params

print(f"student: asnn-goose v11 ({student_params:,} params)")
print(f"projector: ({projector_params:,} params)")
print(f"compression ratio: {compression_ratio:.1f}x")
print(f"")
print(f"v11 architecture:")
print(f"  d_model: {config.d_model}")
print(f"  n_layers: {config.n_layers}")
print(f"  params: ~{student_params // 1_000_000}M")
print(f"")

# v11: compile model if available and enabled
compile_success = False
if USE_TORCH_COMPILE and TORCH_COMPILE_AVAILABLE:
    try:
        print("compiling student model with torch.compile...")
        # Use the compile() method as recommended by PyTorch docs
        student = torch.compile(student, mode='reduce-overhead')
        compile_success = True
        print("compilation successful!")
    except Exception as e:
        print(f"torch.compile failed: {e}")
        print("continuing without compilation")
else:
    print(f"torch.compile skipped (USE_TORCH_COMPILE={USE_TORCH_COMPILE}, available={TORCH_COMPILE_AVAILABLE})")

print(f"")
print(f"speedups active:")
print(f"  gradient checkpointing: {USE_GRADIENT_CHECKPOINTING}")
print(f"  torch.compile: {compile_success}")
print(f"  accumulation_steps: {config.accumulation_steps}")

In [ ]:
# =============================================================================
# cell 15: evaluation functions (same as v9)
# =============================================================================
@torch.no_grad()
def evaluate(model, loader, device, is_gpt2=False):
    model.eval()
    total_loss, total_tokens = 0, 0
    with torch.inference_mode():
      for batch in loader:
        ids = batch[0].to(device, non_blocking=True)
        with torch.cuda.amp.autocast():
            logits = model(ids).logits if is_gpt2 else model(ids)
        loss = F.cross_entropy(logits[:, :-1].reshape(-1, logits.size(-1)), ids[:, 1:].reshape(-1), reduction='sum')
        total_loss += loss.item()
        total_tokens += ids[:, 1:].numel()
    return total_loss / total_tokens

def get_ppl(loss):
    return math.exp(min(loss, 10))

print("evaluation functions defined")

In [ ]:
# =============================================================================
# cell 17: distillation training loop (v12.1 - PROPER CTKD with GRL)
# =============================================================================
def distill_v11(teacher, student, projector, train_loader, val_loader, cfg, device,
               hw_stats, spike_stats):
    """
    v12.1 distillation with PROPER CTKD (Gradient Reversal Layer).
    
    Key differences from v12 (broken):
    - Uses GRL for adversarial min-max optimization
    - Temperature tries to MAXIMIZE loss (via reversed gradients)
    - Student tries to MINIMIZE loss (normal gradients)
    - No manual temperature regularization needed (GRL handles it)
    - Lambda scheduler controls adversarial strength
    
    References:
    - CTKD: https://arxiv.org/abs/2211.16231
    - GRL: https://arxiv.org/abs/1409.7495
    """
    training_logs = {
        'loss_history': [],
        'kl_loss_history': [],
        'align_loss_history': [],
        'ppl_history': [],
        'lr_history': [],
        'temp_history': [],
        'lambda_history': [],  # v12.1: track lambda evolution
        'stage_history': [],
    }

    # =========================================================================
    # v12.1: PROPER CTKD Temperature (with Gradient Reversal Layer)
    # =========================================================================
    if cfg.use_ctkd:
        temp_module = CTKDTemperature(
            tau_min=cfg.tau_min,
            tau_max=cfg.tau_max,
            init=cfg.tau_init
        ).to(device)
        print(f"v12.1: PROPER CTKD with Gradient Reversal Layer")
        print(f"       Temperature bounds: [{cfg.tau_min}, {cfg.tau_max}]")
        print(f"       Initial temp: {cfg.tau_init}")
        print(f"       Lambda warmup: {cfg.lambda_warmup_ratio*100:.0f}% ({int(cfg.distill_steps * cfg.lambda_warmup_ratio)} steps)")
        print(f"       Lambda max: {cfg.lambda_max}")
        print(f"")
        print(f"       Adversarial game:")
        print(f"       - Student gradients: NORMAL (minimize KL loss)")
        print(f"       - Temperature gradients: REVERSED (maximize KL loss)")
    elif cfg.use_learnable_temperature:
        # Legacy (broken) - kept for comparison
        temp_module = LearnableTemperature(init=cfg.temperature).to(device)
        print(f"WARNING: Using legacy LearnableTemperature (caused runaway in v12!)")
    else:
        temp_module = None
        print(f"using fixed temperature: {cfg.temperature}")

    # =========================================================================
    # Setup optimizer
    # =========================================================================
    param_groups = [
        {'params': list(student.parameters()), 'lr': cfg.distill_lr}
    ]
    
    # Add projector params if alignment is enabled
    if cfg.hidden_align_weight > 0:
        param_groups.append({'params': list(projector.parameters()), 'lr': cfg.distill_lr})
        print(f"hidden alignment enabled: weight={cfg.hidden_align_weight}")
    
    # v12.1: Add CTKD temperature params (same LR as model - GRL handles adversarial)
    if temp_module is not None:
        param_groups.append({'params': list(temp_module.parameters()), 'lr': cfg.distill_lr})
        print(f"temperature added to optimizer (same LR as model - GRL controls adversarial)")
    
    # Collect all params for gradient clipping
    all_params = []
    for group in param_groups:
        all_params.extend(group['params'])
    
    # Fused AdamW for speedup
    try:
        optimizer = torch.optim.AdamW(param_groups, weight_decay=0.01, fused=True)
        print("using fused AdamW")
    except TypeError:
        optimizer = torch.optim.AdamW(param_groups, weight_decay=0.01)
        print("fused AdamW not available")
    
    scheduler = get_cosine_schedule_with_warmup(optimizer, cfg.warmup_steps, cfg.distill_steps)
    scaler = torch.cuda.amp.GradScaler()

    hw_stats.start()
    step = 0
    accum_step = 0
    best_val = float('inf')
    current_stage = 1

    accumulation_steps = cfg.accumulation_steps
    effective_batch = cfg.batch_size * accumulation_steps
    print(f"gradient accumulation: {accumulation_steps} steps (effective batch = {effective_batch})")

    pbar = tqdm(total=cfg.distill_steps, desc='distilling (v12.1 - CTKD+GRL)')

    optimizer.zero_grad(set_to_none=True)

    while step < cfg.distill_steps:
        for batch in train_loader:
            if step >= cfg.distill_steps:
                break

            ids = batch[0].to(device, non_blocking=True)

            # =========================================================================
            # v12.1: Compute lambda (adversarial strength) from scheduler
            # =========================================================================
            if cfg.use_ctkd:
                current_lambda = get_lambda(
                    step, cfg.distill_steps,
                    lambda_max=cfg.lambda_max,
                    warmup_ratio=cfg.lambda_warmup_ratio
                )
            else:
                current_lambda = 0.0

            # Get stage params (for legacy infrastructure)
            if cfg.use_progressive_stages:
                stage_params = get_stage_params(step, cfg.distill_steps)
                stage = stage_params['stage']
                align_mult = stage_params['align_mult']
                kd_alpha = stage_params.get('alpha', 0.9)
            else:
                stage = 1
                align_mult = 1.0
                kd_alpha = 1.0
            
            if stage != current_stage:
                print(f"\n  [stage transition] step {step}: stage {current_stage} -> stage {stage}")
                current_stage = stage

            with torch.cuda.amp.autocast():
                # =========================================================================
                # Teacher forward
                # =========================================================================
                with torch.no_grad():
                    if cfg.hidden_align_weight > 0:
                        t_out = teacher(ids, output_hidden_states=True)
                        t_logits = t_out.logits
                        t_hiddens = t_out.hidden_states
                    else:
                        t_logits = teacher(ids).logits

                # =========================================================================
                # Student forward
                # =========================================================================
                student.train()
                if cfg.hidden_align_weight > 0:
                    s_logits, s_hiddens = student(ids, return_hiddens=True)
                else:
                    s_logits = student(ids)

                # =========================================================================
                # v12.1: Get temperature with GRL applied
                # =========================================================================
                if cfg.use_ctkd and temp_module is not None:
                    # Pass current lambda to temperature module
                    # GRL inside temp_module will reverse gradients by -lambda
                    T = temp_module(current_lambda)
                elif temp_module is not None:
                    # Legacy (broken) approach
                    T = temp_module()
                else:
                    T = cfg.temperature

                # =========================================================================
                # KL divergence loss with temperature scaling
                # T² scaling is CRITICAL for proper gradient magnitude
                # =========================================================================
                s_log = F.log_softmax(s_logits / T, dim=-1)
                t_prob = F.softmax(t_logits / T, dim=-1)
                kl_loss = F.kl_div(
                    s_log.view(-1, s_logits.size(-1)),
                    t_prob.view(-1, t_logits.size(-1)),
                    reduction='batchmean'
                ) * (T ** 2)

                # =========================================================================
                # Hidden-state alignment (usually disabled)
                # =========================================================================
                effective_align_weight = cfg.hidden_align_weight * align_mult
                
                if effective_align_weight > 0:
                    align_loss = compute_hidden_alignment_loss(
                        t_hiddens, s_hiddens, projector,
                        teacher_layers=cfg.teacher_n_layers,
                        student_layers=cfg.n_layers
                    )
                else:
                    align_loss = torch.tensor(0.0, device=device)

                # =========================================================================
                # v12.1: NO manual temperature regularization needed!
                # GRL handles the adversarial optimization automatically.
                # The temperature will find optimal difficulty through min-max game.
                # =========================================================================
                
                # Total loss
                kd_weight = kd_alpha if cfg.use_progressive_stages else 1.0
                loss = kd_weight * kl_loss + effective_align_weight * align_loss

                # Scale for gradient accumulation
                loss = loss / accumulation_steps

            scaler.scale(loss).backward()
            accum_step += 1

            if accum_step % accumulation_steps == 0:
                scaler.unscale_(optimizer)
                gn = torch.nn.utils.clip_grad_norm_(all_params, cfg.max_grad_norm)

                if torch.isfinite(gn):
                    scaler.step(optimizer)
                scaler.update()
                scheduler.step()
                optimizer.zero_grad(set_to_none=True)

                hw_stats.record_step(ids.size(0) * accumulation_steps, ids.size(1))
                spike_stats.record(student, step)

                current_lr = optimizer.param_groups[0]['lr']
                current_temp = temp_module.get_temperature() if temp_module is not None else cfg.temperature

                # Track logs
                training_logs['loss_history'].append({'step': step, 'loss': loss.item() * accumulation_steps})
                training_logs['kl_loss_history'].append({'step': step, 'loss': kl_loss.item()})
                training_logs['align_loss_history'].append({'step': step, 'loss': align_loss.item() if isinstance(align_loss, torch.Tensor) else align_loss})
                training_logs['lr_history'].append({'step': step, 'lr': current_lr})
                training_logs['temp_history'].append({'step': step, 'temperature': current_temp})
                training_logs['lambda_history'].append({'step': step, 'lambda': current_lambda})  # v12.1
                training_logs['stage_history'].append({'step': step, 'stage': stage})

                # v12.1: Show lambda in progress bar
                pbar.set_postfix(
                    loss=f"{loss.item() * accumulation_steps:.3f}",
                    kl=f"{kl_loss.item():.3f}",
                    T=f"{current_temp:.2f}",
                    lam=f"{current_lambda:.2f}",  # v12.1: show lambda
                    lr=f"{current_lr:.1e}"
                )
                pbar.update(1)
                step += 1

                if step % cfg.eval_interval == 0:
                    val_loss = evaluate(student, val_loader, device)
                    val_ppl = get_ppl(val_loss)
                    training_logs['ppl_history'].append({'step': step, 'ppl': val_ppl})

                    amps = student.get_amplitudes()
                    amp_str = ', '.join([f"L{i}:{amps[f'layer_{i}']['k']:.2f}" for i in range(min(4, cfg.n_layers))])
                    
                    # v12.1: Show lambda and temperature evolution
                    lambda_str = f", λ={current_lambda:.2f}" if cfg.use_ctkd else ""
                    print(f"\n  step {step}: ppl={val_ppl:.1f}, T={current_temp:.2f}{lambda_str}, amps=[{amp_str}...]")

                    if val_loss < best_val:
                        best_val = val_loss
                        save_dict = {
                            'student': student.state_dict(),
                            'projector': projector.state_dict(),
                        }
                        if temp_module is not None:
                            save_dict['temp_module'] = temp_module.state_dict()
                        torch.save(save_dict, f'{OUTPUT_DIR}/checkpoints/v12_1_best.pt')

    pbar.close()
    return training_logs

print("distillation function defined (v12.1 - PROPER CTKD with GRL)")

In [ ]:
# =============================================================================
# cell 18: run distillation (v12.1 - PROPER CTKD with GRL)
# =============================================================================
print("="*60)
print("starting distillation (v12.1 - PROPER CTKD with GRL)")
print("="*60)
print(f"  architecture: {config.d_model}d × {config.n_layers}L (~22M params)")
print(f"")
print(f"v12.1 CTKD Configuration:")
print(f"  use_ctkd: {config.use_ctkd}")
print(f"  temperature bounds: [{config.tau_min}, {config.tau_max}]")
print(f"  initial temperature: {config.tau_init}")
print(f"  lambda_max: {config.lambda_max}")
print(f"  lambda warmup: {config.lambda_warmup_ratio*100:.0f}% ({int(config.distill_steps * config.lambda_warmup_ratio)} steps)")
print(f"")
print(f"v12.1 KEY DIFFERENCES from v12 (broken):")
print(f"  - Uses Gradient Reversal Layer for ADVERSARIAL min-max game")
print(f"  - Temperature tries to MAXIMIZE loss (find hardest difficulty)")
print(f"  - Student tries to MINIMIZE loss (learn from challenge)")
print(f"  - NO manual regularization needed (GRL handles it)")
print(f"")
print(f"  target: PPL < 500 (improve on v10's 514.5)")
print("")

# Instantiate collectors (required for distill_v11)
hw_stats = HardwareStatsCollector()
spike_stats = SpikeStatsCollector(config.n_layers)

distill_logs = distill_v11(
    teacher, student, projector,
    train_loader, val_loader,
    config, DEVICE,
    hw_stats, spike_stats
)

print("\ndistillation complete (v12.1 - PROPER CTKD with GRL)")

In [ ]:
# =============================================================================
# cell 20: lora implementation (same as v9)
# =============================================================================
class LoRALinear(nn.Module):
    """lora adapter for linear layers."""

    def __init__(self, in_features, out_features, rank=8, alpha=16.0):
        super().__init__()
        self.scaling = alpha / rank
        self.lora_A = nn.Parameter(torch.zeros(rank, in_features))
        self.lora_B = nn.Parameter(torch.zeros(out_features, rank))
        nn.init.kaiming_uniform_(self.lora_A, a=math.sqrt(5))

    def forward(self, x):
        return (x @ self.lora_A.T @ self.lora_B.T) * self.scaling


def apply_lora(model, rank=8, alpha=16.0, targets=['key_proj', 'value_proj']):
    """apply lora adapters to specified modules."""
    lora_modules = {}
    for name, module in model.named_modules():
        if any(t in name for t in targets) and isinstance(module, nn.Linear):
            lora = LoRALinear(module.in_features, module.out_features, rank, alpha).to(next(module.parameters()).device)
            lora_modules[name] = lora
            orig_forward = module.forward
            def make_forward(orig, lora_mod):
                def forward(x):
                    return orig(x) + lora_mod(x)
                return forward
            module.forward = make_forward(orig_forward, lora)
    print(f"lora: {len(lora_modules)} modules, rank={rank}")
    return lora_modules

print("lora defined")

In [ ]:
# =============================================================================
# cell 21: ttt with lora (same as v9)
# =============================================================================
print("="*60)
print("phase 2: test-time training with lora")
print("="*60)

for p in student.parameters():
    p.requires_grad = False

lora_modules = apply_lora(student, config.lora_rank, config.lora_alpha)
lora_params = sum(p.numel() for m in lora_modules.values() for p in m.parameters())

pre_ttt_loss = evaluate(student, val_loader, DEVICE)
pre_ttt_ppl = get_ppl(pre_ttt_loss)
print(f"\npre-ttt ppl: {pre_ttt_ppl:.2f}")

lora_opt = torch.optim.AdamW([p for m in lora_modules.values() for p in m.parameters()], lr=config.ttt_lr)
ttt_logs = {'loss_history': []}
student.train()

for step, batch in enumerate(val_loader):
    if step >= config.ttt_steps:
        break
    ids = batch[0].to(DEVICE)
    with torch.cuda.amp.autocast():
        loss = F.cross_entropy(student(ids)[:, :-1].reshape(-1, config.vocab_size), ids[:, 1:].reshape(-1))
    lora_opt.zero_grad()
    loss.backward()
    lora_opt.step()
    ttt_logs['loss_history'].append({'step': step, 'loss': loss.item()})
    if step % 20 == 0:
        print(f"  ttt {step}: loss={loss.item():.4f}")

post_ttt_loss = evaluate(student, val_loader, DEVICE)
post_ttt_ppl = get_ppl(post_ttt_loss)
print(f"\npost-ttt ppl: {post_ttt_ppl:.2f}")
print(f"ttt improvement: {pre_ttt_ppl - post_ttt_ppl:.1f} ppl")

In [ ]:
# =============================================================================
# cell 22: final evaluation (v12.1 - Proper CTKD with GRL)
# =============================================================================
print("="*60)
print("final evaluation (v12.1 - CTKD with Gradient Reversal Layer)")
print("="*60)

teacher_loss = evaluate(teacher, val_loader, DEVICE, is_gpt2=True)
teacher_ppl = get_ppl(teacher_loss)
student_loss = evaluate(student, val_loader, DEVICE)
student_ppl = get_ppl(student_loss)

# v12.1: Get final temperature and lambda from CTKD
final_temp = distill_logs['temp_history'][-1]['temperature'] if distill_logs['temp_history'] else config.tau_init
final_lambda = distill_logs['temp_history'][-1].get('lambda', config.lambda_max) if distill_logs['temp_history'] else config.lambda_max

# VRAM logging
vram_peak_gb = torch.cuda.max_memory_allocated() / 1e9 if torch.cuda.is_available() else 0

print(f"")
print(f"{'model':<30} {'ppl':>10} {'params':>15}")
print("-" * 55)
print(f"{'gpt-2 (teacher)':<30} {teacher_ppl:>10.2f} {teacher_params:>15,}")
print(f"{'asnn-goose v12.1 (student)':<30} {student_ppl:>10.2f} {student_params:>15,}")
print("-" * 55)
print(f"{'compression':<30} {compression_ratio:>10.1f}x")
print(f"{'ppl gap':<30} {student_ppl - teacher_ppl:>10.2f}")
print(f"{'spike density':<30} {student.get_avg_spike_density():>10.3f}")
print(f"{'VRAM peak':<30} {vram_peak_gb:>10.2f}GB")
print(f"{'final temperature':<30} {final_temp:>10.2f}")
print(f"{'final lambda (GRL)':<30} {final_lambda:>10.3f}")
print("")
print("CTKD Implementation:")
print(f"  tau range: [{config.tau_min:.1f}, {config.tau_max:.1f}]")
print(f"  lambda warmup ratio: {config.lambda_warmup_ratio:.0%}")
print(f"  GRL: Gradient Reversal Layer for adversarial min-max")
print("")
print("version comparison:")
print(f"  v6: 627.3 PPL (baseline)")
print(f"  v7: 1655 PPL (regression!)")
print(f"  v8: 559 PPL (fixed)")
print(f"  v9: 541.7 PPL (capacity increase)")
print(f"  v10: 514.5 PPL (320d/5L baseline)")
print(f"  v11: 512.67 PPL (channel-wise, WITH reg)")
print(f"  v11.1: 512.04 PPL (channel-wise, NO reg)")
print(f"  v12: FAILED (temp runaway without GRL)")
print(f"  v12.1: {student_ppl:.2f} PPL (CTKD+GRL, T={final_temp:.2f}, λ={final_lambda:.3f})")
if student_ppl < 500:
    print(f"  v12.1 TARGET MET! PPL < 500")
elif student_ppl < 512.04:
    print(f"  v12.1 beats v11.1 by {512.04 - student_ppl:.1f} PPL")
else:
    print(f"  WARNING: v12.1 did not improve over v11.1")


In [ ]:
# =============================================================================
# cell 23: visualization
# =============================================================================
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# distillation loss
d_steps = [l['step'] for l in distill_logs['loss_history']]
d_losses = [l['loss'] for l in distill_logs['loss_history']]
kl_losses = [l['loss'] for l in distill_logs['kl_loss_history']]
axes[0,0].plot(d_steps, d_losses, label='total', alpha=0.8)
axes[0,0].plot(d_steps, kl_losses, label='kl', alpha=0.7)
axes[0,0].set_xlabel('step')
axes[0,0].set_ylabel('loss')
axes[0,0].set_title('distillation loss (v11)')
axes[0,0].legend()

# validation ppl
p_steps = [l['step'] for l in distill_logs['ppl_history']]
p_ppls = [l['ppl'] for l in distill_logs['ppl_history']]
axes[0,1].plot(p_steps, p_ppls, 'orange', marker='o')
axes[0,1].axhline(y=teacher_ppl, color='green', linestyle='--', label=f'teacher ({teacher_ppl:.1f})')
axes[0,1].axhline(y=627.3, color='blue', linestyle=':', label='v6 (627.3)')
axes[0,1].axhline(y=541.7, color='purple', linestyle=':', label='v9 (541.7)')
axes[0,1].axhline(y=480, color='red', linestyle='--', label='v11 target (480)')
axes[0,1].set_xlabel('step')
axes[0,1].set_ylabel('ppl')
axes[0,1].set_title('validation ppl')
axes[0,1].legend()

# lr schedule
lr_steps = [l['step'] for l in distill_logs['lr_history']]
lr_vals = [l['lr'] for l in distill_logs['lr_history']]
axes[0,2].plot(lr_steps, lr_vals, 'purple')
axes[0,2].axvline(x=config.warmup_steps, color='gray', linestyle='--', label=f'warmup ({config.warmup_steps})')
axes[0,2].set_xlabel('step')
axes[0,2].set_ylabel('lr')
axes[0,2].set_title('learning rate')
axes[0,2].legend()

# spike density + amplitudes (first 4 layers)
spike_summary = spike_stats.get_summary()
layers = [f'layer_{i}' for i in range(min(4, config.n_layers))]
k_dens = [spike_summary['per_layer'][l]['k_final'] for l in layers]
v_dens = [spike_summary['per_layer'][l]['v_final'] for l in layers]
k_amps = [spike_summary['per_layer'][l]['k_amp_final'] for l in layers]
v_amps = [spike_summary['per_layer'][l]['v_amp_final'] for l in layers]

x = np.arange(len(layers))
axes[1,0].bar(x - 0.2, k_dens, 0.4, label='k density')
axes[1,0].bar(x + 0.2, v_dens, 0.4, label='v density')
ax2 = axes[1,0].twinx()
ax2.plot(x, k_amps, 'r-o', label='k amp')
ax2.plot(x, v_amps, 'b-s', label='v amp')
axes[1,0].set_xlabel('layer')
axes[1,0].set_ylabel('density')
ax2.set_ylabel('amplitude')
axes[1,0].set_title(f'spike density & amps (first 4/{config.n_layers} layers)')
axes[1,0].legend(loc='upper left')
ax2.legend(loc='upper right')

# ttt loss
t_steps = [l['step'] for l in ttt_logs['loss_history']]
t_losses = [l['loss'] for l in ttt_logs['loss_history']]
axes[1,1].plot(t_steps, t_losses, 'red')
axes[1,1].set_xlabel('step')
axes[1,1].set_ylabel('ce loss')
axes[1,1].set_title('ttt with lora')

# version comparison
versions = ['v6', 'v7', 'v8', 'v9', 'v11']
t_ppls = [44.6, 44.6, 44.6, 44.6, teacher_ppl]
s_ppls = [627.3, 1655, 559, 541.7, student_ppl]
x = np.arange(len(versions))
axes[1,2].bar(x - 0.2, t_ppls, 0.4, label='teacher', alpha=0.7)
axes[1,2].bar(x + 0.2, s_ppls, 0.4, label='student', alpha=0.7)
axes[1,2].axhline(y=480, color='red', linestyle='--', label='v11 target', alpha=0.7)
axes[1,2].set_xticks(x)
axes[1,2].set_xticklabels(versions)
axes[1,2].set_ylabel('ppl')
axes[1,2].set_title('version comparison')
axes[1,2].legend()
axes[1,2].set_yscale('log')

plt.tight_layout()
figure_path = f'{OUTPUT_DIR}/figures/v11_training_{RUN_TIMESTAMP}.png'
plt.savefig(figure_path, dpi=300, bbox_inches='tight')
plt.show()
print(f"saved: {figure_path}")

In [ ]:
# =============================================================================
# cell 24: build results dict (v12.1 - Proper CTKD with GRL)
# =============================================================================
print("building results (v12.1 - CTKD with Gradient Reversal Layer)...")

with open(figure_path, 'rb') as f:
    figure_base64 = base64.b64encode(f.read()).decode('utf-8')

# v12.1: Extract final lambda
final_lambda = distill_logs['temp_history'][-1].get('lambda', config.lambda_max) if distill_logs['temp_history'] else config.lambda_max

results = {
    'version': 'v12.1',
    'timestamp': datetime.now().isoformat(),
    'run_id': RUN_TIMESTAMP,
    'platform': PLATFORM,
    'description': 'CTKD with Gradient Reversal Layer - adversarial temperature learning',

    'v12.1_design': {
        'principle': 'CTKD: Adversarial min-max optimization via GRL',
        'innovation': 'Gradient Reversal Layer makes temperature MAXIMIZE KL while student MINIMIZES',
        'rationale': 'Proper CTKD (ArXiv 2211.16231) requires adversarial training, not simple regularization',
        'why_v12_failed': 'v12 used simple regularization - optimizer pushed T to max for easy KL',
        'techniques': {
            'ctkd_with_grl': 'ENABLED - Gradient Reversal Layer for adversarial min-max (v12.1 KEY)',
            'lambda_scheduling': f'Cosine warmup 0->{config.lambda_max} with {config.lambda_warmup_ratio:.0%} warmup',
            'sigmoid_bounding': f'T bounded to [{config.tau_min}, {config.tau_max}] via sigmoid (smooth gradients)',
            'no_manual_reg': 'GRL eliminates need for manual temperature regularization',
            'progressive_stages': 'DISABLED',
            'channel_wise_spikes': 'DISABLED (structural symmetry issue)',
        },
        'grl_mechanism': {
            'forward_pass': 'Identity: GRL(x) = x',
            'backward_pass': 'Negation: dGRL/dx = -lambda',
            'effect': 'Temperature gradients reversed -> T maximizes KL loss',
        },
        'temperature_config': {
            'tau_min': config.tau_min,
            'tau_max': config.tau_max,
            'tau_init': config.tau_init,
            'lambda_max': config.lambda_max,
            'lambda_warmup_ratio': config.lambda_warmup_ratio,
        },
        'architecture': {
            'd_model': 320,
            'n_layers': 5,
            'params': '~22M',
        },
        'speedups': {
            'gradient_checkpointing': USE_GRADIENT_CHECKPOINTING,
            'torch_compile': compile_success,
            'fused_optimizer': True,
            'accumulation_steps': config.accumulation_steps,
        },
        'unchanged': [
            'hidden_align_weight: 0.0',
            'warmup_steps: 50',
            'distill_steps: 3000',
        ],
    },

    'architecture': {
        'teacher': {'name': 'gpt2', 'params': teacher_params},
        'student': {
            'name': 'asnn-goose-v12.1',
            'd_model': config.d_model,
            'n_layers': config.n_layers,
            'params': student_params,
        },
        'projector_params': projector_params,
        'compression_ratio': compression_ratio,
        'vram_peak_gb': vram_peak_gb,
    },

    'training_config': {
        'distill_steps': config.distill_steps,
        'tau_min': config.tau_min,
        'tau_max': config.tau_max,
        'tau_init': config.tau_init,
        'final_temperature': final_temp,
        'lambda_max': config.lambda_max,
        'lambda_warmup_ratio': config.lambda_warmup_ratio,
        'final_lambda': final_lambda,
        'hidden_align_weight': config.hidden_align_weight,
        'warmup_steps': config.warmup_steps,
        'batch_size': config.batch_size,
        'accumulation_steps': config.accumulation_steps,
        'effective_batch': config.batch_size * config.accumulation_steps,
        'distill_lr': config.distill_lr,
        'max_grad_norm': config.max_grad_norm,
    },

    'results': {
        'teacher_ppl': teacher_ppl,
        'student_ppl': student_ppl,
        'ppl_gap': student_ppl - teacher_ppl,
        'spike_density': student.get_avg_spike_density(),
        'amplitudes': student.get_amplitudes(),
        'final_temperature': final_temp,
        'final_lambda': final_lambda,
        'target_met': student_ppl < 500,
    },

    'training_curves': {
        'loss_history': distill_logs['loss_history'],
        'kl_loss_history': distill_logs['kl_loss_history'],
        'align_loss_history': distill_logs['align_loss_history'],
        'ppl_history': distill_logs['ppl_history'],
        'lr_history': distill_logs['lr_history'],
        'temp_history': distill_logs['temp_history'],  # v12.1: includes temperature AND lambda
    },

    'hardware_stats': hw_stats.get_summary(),
    'spike_analysis': spike_stats.get_summary(),

    'ttt': {
        'lora_params': lora_params,
        'pre_ppl': pre_ttt_ppl,
        'post_ppl': post_ttt_ppl,
        'improvement': pre_ttt_ppl - post_ttt_ppl,
        'loss_history': ttt_logs['loss_history'],
    },

    'comparison': {
        'v6': {'student_ppl': 627.3, 'note': 'baseline'},
        'v7': {'student_ppl': 1655, 'note': 'regression (align=1.0, T=4)'},
        'v8': {'student_ppl': 559, 'note': 'fixed defaults (align=0, T=2)'},
        'v9': {'student_ppl': 541.7, 'note': 'capacity increase (320d, 5L)'},
        'v10': {'student_ppl': 514.5, 'note': '320d/5L baseline'},
        'v11': {'student_ppl': 512.67, 'note': 'channel-wise WITH reg (bug)'},
        'v11.1': {'student_ppl': 512.04, 'note': 'channel-wise NO reg (symmetry issue)'},
        'v12': {'student_ppl': 'FAILED', 'note': 'temp runaway without GRL'},
        'v12.1': {'student_ppl': student_ppl, 'note': f'CTKD+GRL (T={final_temp:.2f}, L={final_lambda:.3f})'},
    },

    'figures': {
        'training_plot': {
            'filename': f'v12.1_training_{RUN_TIMESTAMP}.png',
            'base64': figure_base64,
        }
    },

    # validation_tests will be added in cell 26
}

print("results dict built (validation_tests pending)")
print(f"  version: v12.1 (CTKD with GRL)")
print(f"  final_temperature: {final_temp:.2f}")
print(f"  final_lambda: {final_lambda:.3f}")


In [ ]:
# =============================================================================
# cell 25: validation tests (v12.1 - 10 tests with proper CTKD)
# =============================================================================
print("="*60)
print("validation tests (v12.1 - CTKD with Gradient Reversal Layer)")
print("="*60)

test_results = {}

# 1. teacher pre-trained
print("\n[1] teacher pre-trained")
test_results['teacher_pretrained'] = teacher_ppl < 50
print(f"  teacher ppl: {teacher_ppl:.2f} - {'pass' if test_results['teacher_pretrained'] else 'fail'}")

# 2. student learned (v12.1 target: <500)
print("\n[2] student learned language")
test_results['student_learned'] = student_ppl < 627
print(f"  student ppl: {student_ppl:.2f} - {'pass' if test_results['student_learned'] else 'fail'} (target: < 627)")

# 3. ternary activations (scalar amplitudes)
print("\n[3] ternary activations")
student.eval()
with torch.no_grad():
    test_ids = next(iter(val_loader))[0].to(DEVICE)
    layer = student.layers[0]['rec']
    x = student.embed(test_ids) + student.pos_embed(torch.arange(test_ids.size(1), device=DEVICE).unsqueeze(0))
    x_norm = layer.ln(x)
    prev_x = F.pad(x_norm[:, :-1, :], (0, 0, 1, 0))
    xk = x_norm * layer.time_mix_k + prev_x * (1 - layer.time_mix_k)
    k_spike = layer.k_spike(layer.key_proj(xk))

    # For scalar amplitudes, values should be {-amp, 0, +amp} = 3 unique
    # Round to 4 decimal places to handle floating point
    unique_vals = len(set([round(v, 4) for v in k_spike.unique().cpu().tolist()]))
    test_results['ternary'] = unique_vals <= 3
    print(f"  unique values: {unique_vals} - {'pass' if test_results['ternary'] else 'fail'}")

# 4. gradient flow
print("\n[4] gradient flow (STE)")
_spike = TrainableTernarySpike().to(DEVICE)
_x = torch.randn(2, 16, 64, device=DEVICE, requires_grad=True)
_spike(_x).sum().backward()
test_results['gradient'] = _x.grad is not None and _x.grad.abs().sum() > 0
print(f"  {'pass' if test_results['gradient'] else 'fail'}")

# 5. spike density
print("\n[5] spike density in range")
density = student.get_avg_spike_density()
test_results['density'] = 0.1 < density < 0.9
print(f"  density: {density:.3f} - {'pass' if test_results['density'] else 'fail'}")

# 6. lora applied
print("\n[6] lora applied")
test_results['lora'] = len(lora_modules) > 0
print(f"  modules: {len(lora_modules)} - {'pass' if test_results['lora'] else 'fail'}")

# 7. improvement over v6 baseline
print("\n[7] beats v6 baseline")
test_results['beats_v6'] = student_ppl < 627.3
print(f"  v6: 627.3, v12.1: {student_ppl:.2f} - {'pass' if test_results['beats_v6'] else 'fail'}")

# =============================================================================
# [8] v12.1: Amplitudes learned (scalar, not channel-wise)
# v12.1 uses TrainableTernarySpike (scalar amplitude) since channel-wise disabled
# =============================================================================
print("\n[8] amplitudes learned (scalar)")
amps = student.get_amplitudes()
amp_vals = []
for layer_idx, amp_dict in amps.items():
    amp_vals.extend([amp_dict['k'], amp_dict['v']])

# Scalar amplitudes should deviate from 1.0 (initial value)
amp_deviation = max(abs(a - 1.0) for a in amp_vals)
test_results['amplitudes_learned'] = amp_deviation > 0.01  # Should move from 1.0
amp_strs = [f"L{i}:{amps[f'layer_{i}']['k']:.3f}" for i in range(config.n_layers)]
print(f"  amplitudes: {amp_strs}")
print(f"  max deviation from 1.0: {amp_deviation:.4f}")
print(f"  {'pass' if test_results['amplitudes_learned'] else 'fail'} - amplitudes should deviate from 1.0")

# 9. amplitude health check
print("\n[9] amplitude health")
all_healthy = True
for layer_idx, amp_dict in amps.items():
    k_amp, v_amp = amp_dict['k'], amp_dict['v']
    if not (0.3 < k_amp < 3.0) or not (0.3 < v_amp < 3.0):
        print(f"  WARNING: {layer_idx} unhealthy: k={k_amp:.3f}, v={v_amp:.3f}")
        all_healthy = False
test_results['amplitude_health'] = all_healthy
print(f"  {'pass' if all_healthy else 'fail'} - all amplitudes in [0.3, 3.0]")

# =============================================================================
# [10] v12.1 KEY TEST: CTKD with Gradient Reversal Layer
# This is THE key test for v12.1 - verifies proper adversarial temperature learning
# =============================================================================
print("\n[10] CTKD with GRL (v12.1 KEY TEST)")
if config.use_ctkd and 'temp_history' in distill_logs:
    temps = [h['temperature'] for h in distill_logs['temp_history']]
    lambdas = [h.get('lambda', 0) for h in distill_logs['temp_history']]

    if len(temps) > 100:
        # ===== CTKD-SPECIFIC CHECKS =====

        # 1. Lambda schedule worked (should go from 0 to ~lambda_max)
        warmup_steps = int(config.distill_steps * config.lambda_warmup_ratio)
        warmup_lambdas = lambdas[:min(warmup_steps // 10, len(lambdas))]  # Samples during warmup
        late_lambdas = lambdas[int(len(lambdas) * 0.8):]  # Last 20%

        lambda_was_zero_early = len(warmup_lambdas) > 0 and all(l < 0.01 for l in warmup_lambdas)
        lambda_increased = len(late_lambdas) > 0 and np.mean(late_lambdas) > 0.5

        print(f"  lambda schedule:")
        print(f"    early (warmup): {np.mean(warmup_lambdas) if warmup_lambdas else 0:.4f} (should be ~0)")
        print(f"    late: {np.mean(late_lambdas) if late_lambdas else 0:.4f} (should be ~{config.lambda_max})")
        print(f"    warmup correct: {lambda_was_zero_early}")
        print(f"    increased: {lambda_increased}")

        # 2. Temperature stayed in bounds [tau_min, tau_max]
        min_temp, max_temp = min(temps), max(temps)
        temp_in_bounds = min_temp >= config.tau_min - 0.01 and max_temp <= config.tau_max + 0.01
        print(f"  temperature bounds: [{min_temp:.3f}, {max_temp:.3f}]")
        print(f"    expected: [{config.tau_min}, {config.tau_max}]")
        print(f"    in bounds: {temp_in_bounds}")

        # 3. Temperature did NOT runaway to max (unlike v12)
        final_temp_val = temps[-1]
        not_at_max = final_temp_val < config.tau_max - 0.1  # Not stuck at upper bound
        not_at_min = final_temp_val > config.tau_min + 0.1  # Not stuck at lower bound
        print(f"  final temperature: {final_temp_val:.3f}")
        print(f"    not at max ({config.tau_max}): {not_at_max}")
        print(f"    not at min ({config.tau_min}): {not_at_min}")

        # 4. Temperature evolved meaningfully (changed during training)
        temp_range = max_temp - min_temp
        temp_evolved = temp_range > 0.05  # Should have some movement
        print(f"  temperature evolution range: {temp_range:.4f}")
        print(f"    evolved: {temp_evolved}")

        # 5. Temperature stabilized (low variance in last 20%)
        late_temps = temps[int(len(temps) * 0.8):]
        temp_variance = np.std(late_temps)
        temp_stable = temp_variance < 0.5  # Tighter bound for CTKD
        print(f"  late temperature variance: {temp_variance:.4f}")
        print(f"    stable: {temp_stable}")

        # OVERALL CTKD TEST
        # Pass requires:
        # - Lambda schedule worked (was 0 during warmup, increased after)
        # - Temperature stayed in sigmoid bounds
        # - Temperature didn't runaway (GRL prevented max seeking)
        # - Temperature evolved and stabilized
        ctkd_working = (
            lambda_increased and  # Lambda actually increased
            temp_in_bounds and    # Sigmoid bounding worked
            not_at_max and        # GRL prevented runaway
            temp_stable           # Training converged
        )

        test_results['ctkd_temperature'] = ctkd_working
        print(f"\n  CTKD RESULT: {'PASS' if ctkd_working else 'FAIL'}")

        if not lambda_increased:
            print(f"  ISSUE: Lambda did not increase (GRL not active?)")
        if not temp_in_bounds:
            print(f"  ISSUE: Temperature escaped sigmoid bounds")
        if not not_at_max:
            print(f"  ISSUE: Temperature stuck at max (GRL failed to prevent runaway)")
        if not temp_stable:
            print(f"  ISSUE: Temperature still oscillating (training not converged)")

    else:
        test_results['ctkd_temperature'] = False
        print(f"  not enough data points ({len(temps)}) - fail")
else:
    test_results['ctkd_temperature'] = False  # FAIL if CTKD disabled in v12.1
    print(f"  CTKD disabled - FAIL (v12.1 requires this feature)")

print("\n" + "="*60)
passed = sum(1 for v in test_results.values() if v)
print(f"results: {passed}/{len(test_results)} passed")
if student_ppl < 500:
    print(f"v12.1 TARGET MET! PPL {student_ppl:.2f} < 500")
elif student_ppl < 512.04:
    print(f"v12.1 improved from v11.1: {512.04 - student_ppl:.1f} PPL reduction")
else:
    print(f"v12.1 did NOT improve from v11.1 (512.04)")


In [ ]:
# =============================================================================
# cell 26: FINAL save with validation_tests (v11 bug fix)
# =============================================================================
print("="*60)
print("FINAL SAVE (includes validation_tests)")
print("="*60)

# Add validation_tests to results
results['validation_tests'] = test_results

# Save final results.json with ALL data
results_path = f'{OUTPUT_DIR}/results/results_{RUN_TIMESTAMP}.json'
with open(results_path, 'w') as f:
    json.dump(results, f, indent=2, default=str)

print(f"saved: {results_path}")
print(f"size: {os.path.getsize(results_path) / 1024:.1f} KB")
print(f"")
print(f"validation_tests included: {list(test_results.keys())}")

# v11: auto-download AFTER final save
print("")
print("auto-download")
if IS_COLAB:
    try:
        from google.colab import files
        files.download(results_path)
        files.download(figure_path)
        print("downloads started!")
    except Exception as e:
        print(f"download failed: {e}")
elif IS_KAGGLE:
    print(f"kaggle: {results_path}")
else:
    print(f"local: {results_path}")